- **Pydantic** is a Python library for data validation and settings management using Python type annotations.
- It is build by rust so its super fast and it performs validation and type conversion super easy and faster.
- It ensures that data types are correct by parsing and validating input data (e.g., from APIs or user input).
- Built on Python’s `dataclasses` and `typing` modules for cleaner, more readable code.
- Automatically raises errors if input data doesn't conform to expected types or constraints.
- Commonly used with **FastAPI**, **ORMs**, and **configuration management**.


In [3]:
from dataclasses import dataclass

@dataclass
class Product:
    name: str
    price: float
    in_stock: bool = True

item = Product("Chocolate", 2.99)
print(item)

#data validation is not done automatically in dataclasses
item2 = Product(1223, '222')
print(item2)

Product(name='Chocolate', price=2.99, in_stock=True)
Product(name=1223, price='222', in_stock=True)


In [15]:
from pydantic import BaseModel, ValidationError
from typing import Optional,List


class Product(BaseModel):
    name: str
    price: float
    stock_list : Optional[List[str]] = []
    in_stock: Optional[bool] = False  # Now it's optional

item = Product(name="Chocolate", price='2',stock_list=["item1", "item2"])
print(item)


name='Chocolate' price=2.0 stock_list=['item1', 'item2'] in_stock=False


In [16]:
item1 = Product(name="Chocolate", price=2.99,in_stock=True,stock_list=("item1", "item2",1))
print(item1)

ValidationError: 1 validation error for Product
stock_list.2
  Input should be a valid string [type=string_type, input_value=1, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/string_type

In [17]:
item = Product(name="Chocolate", price="2",in_stock=False)
print(item)


name='Chocolate' price=2.0 stock_list=[] in_stock=False


In [18]:
from pydantic import BaseModel, StrictFloat

class Product(BaseModel):
    name: str
    price: StrictFloat
    in_stock: bool = True

item = Product(name="Chocolate", price=2.99)
print(item)


name='Chocolate' price=2.99 in_stock=True


In [19]:
item = Product(name="Chocolate", price="2",in_stock=False)
print(item)


ValidationError: 1 validation error for Product
price
  Input should be a valid number [type=float_type, input_value='2', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/float_type

In [20]:
from pydantic import BaseModel
from typing import Optional

class Category(BaseModel):
    id: int
    name: str

class Product(BaseModel):
    name: str
    price: float
    in_stock: Optional[bool] = True
    category: Category  # Nested model

# Creating a product with a nested category
item = Product(
    name="Chocolate",
    price=2.99,
    category=Category(id=1, name="Snacks")
)

print(item)


name='Chocolate' price=2.99 in_stock=True category=Category(id=1, name='Snacks')


In [21]:
from pydantic import BaseModel, Field

class Product(BaseModel):
    name: str = Field(..., min_length=2, max_length=50, description="Product name")
    price: float = Field(..., gt=0, description="Must be greater than 0")
    stock: int = Field(default=0, ge=0, description="Non-negative integer")


In [22]:
product = Product(name="Apple", price=1.25)
print(product)


name='Apple' price=1.25 stock=0


In [23]:
product.model_json_schema()

{'properties': {'name': {'description': 'Product name',
   'maxLength': 50,
   'minLength': 2,
   'title': 'Name',
   'type': 'string'},
  'price': {'description': 'Must be greater than 0',
   'exclusiveMinimum': 0,
   'title': 'Price',
   'type': 'number'},
  'stock': {'default': 0,
   'description': 'Non-negative integer',
   'minimum': 0,
   'title': 'Stock',
   'type': 'integer'}},
 'required': ['name', 'price'],
 'title': 'Product',
 'type': 'object'}

In [24]:
from pydantic import BaseModel, field_validator

class User(BaseModel):
    age: int

    @field_validator('age')
    def age_must_be_positive(cls, v):
        if v <= 0:
            raise ValueError('Age must be positive')
        return v
user = User(age=-25)
print(user)


ValidationError: 1 validation error for User
age
  Value error, Age must be positive [type=value_error, input_value=-25, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

In [25]:
user = User(age=-25)
print(user)

ValidationError: 1 validation error for User
age
  Value error, Age must be positive [type=value_error, input_value=-25, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

In [26]:
from pydantic import BaseModel, model_validator
from datetime import date

class Booking(BaseModel):
    start_date: date
    end_date: date

    @model_validator(mode='after')
    def check_dates(self):
        if self.end_date < self.start_date:
            raise ValueError('End date must be after start date')
        return self


# -----------------------
# Valid Input
# -----------------------
try:
    booking = Booking(start_date='2025-05-01', end_date='2025-05-10')
    print("Valid booking:", booking)
except ValidationError as e:
    print("Validation Error:\n", e)

# -----------------------
# Invalid Input
# -----------------------
try:
    booking = Booking(start_date='2025-05-10', end_date='2025-05-01')
    print("Booking created:", booking)
except ValidationError as e:
    print("Validation Error:\n", e)


Valid booking: start_date=datetime.date(2025, 5, 1) end_date=datetime.date(2025, 5, 10)
Validation Error:
 1 validation error for Booking
  Value error, End date must be after start date [type=value_error, input_value={'start_date': '2025-05-1...end_date': '2025-05-01'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error


In [27]:
from pydantic import BaseModel, model_validator
from datetime import date, datetime

class Event(BaseModel):
    name: str
    start: date
    end: date

    # Before: convert datetime to date if needed
    @model_validator(mode='before')
    @classmethod
    def convert_dates(cls, data):
        if isinstance(data.get("start"), datetime):
            data["start"] = data["start"].date()
        if isinstance(data.get("end"), datetime):
            data["end"] = data["end"].date()
        return data

    # After: ensure start < end
    @model_validator(mode='after')
    def check_date_order(self):
        if self.start >= self.end:
            raise ValueError("Start date must be before end date")
        return self

# ✅ Valid input with datetime objects
event = Event(name="Launch", start=datetime(2025, 6, 1, 9), end=datetime(2025, 6, 3, 17))
print(event)

# ❌ Invalid: start date after end date
try:
    bad_event = Event(name="Bug", start=date(2025, 6, 5), end=date(2025, 6, 4))
except Exception as e:
    print("\nValidation Error:", e)


name='Launch' start=datetime.date(2025, 6, 1) end=datetime.date(2025, 6, 3)

Validation Error: 1 validation error for Event
  Value error, Start date must be before end date [type=value_error, input_value={'name': 'Bug', 'start': ...tetime.date(2025, 6, 4)}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error


In [28]:
from pydantic import BaseModel, ValidationError
from typing import Optional,List
from pydantic import BaseModel, Field

class Product(BaseModel):
    name: str
    price: float = Field(strict=True, description="Price of the product")

item = Product(name="Chocolate", price='2.99',stock_list=["item1", "item2"])
item1 = Product(name="Chocolate", price=2.99,stock_list=["item1", "item2"])
print(item)
print(item1)

ValidationError: 1 validation error for Product
price
  Input should be a valid number [type=float_type, input_value='2.99', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/float_type

In [29]:

from pydantic import BaseModel, Field

class Foo(BaseModel):
    positive: int = Field(gt=0)
    non_negative: int = Field(ge=0)
    negative: int = Field(lt=0)
    non_positive: int = Field(le=0)
    even: int = Field(multiple_of=2)
    love_for_pydantic: float = Field(allow_inf_nan=True)

foo = Foo(
    positive=1,
    non_negative=0,
    negative=-1,
    non_positive=0,
    even=2,
    love_for_pydantic=float('inf'),
)
print(foo)

positive=1 non_negative=0 negative=-1 non_positive=0 even=2 love_for_pydantic=inf


In [30]:
from pydantic import BaseModel, Field


class Foo(BaseModel):
    short: str = Field(min_length=3)
    long: str = Field(max_length=10)
    regex: str = Field(pattern=r'^\d*$')  


foo = Foo(short='foo', long='foobarbaz', regex='123')
print(foo)

short='foo' long='foobarbaz' regex='123'


In [31]:
from decimal import Decimal

from pydantic import BaseModel, Field


class Foo(BaseModel):
    precise: Decimal = Field(max_digits=5, decimal_places=2)


foo = Foo(precise=Decimal('123.45'))
print(foo)
#> precise=Decimal('123.45')

precise=Decimal('123.45')


In [32]:
from pydantic import BaseModel, Field

class Product(BaseModel):
    name: str = Field(..., min_length=2, max_length=50, description="Product name")
    price: float = Field(..., gt=0, description="Must be greater than 0")
    stock: int = Field(default=0, ge=0, description="Non-negative integer")

product = Product(name="Apple", price=1.25)
print(product)

name='Apple' price=1.25 stock=0


In [34]:
from pydantic import BaseModel, field_validator

class User(BaseModel):
    age: int

    @field_validator('age')
    def age_must_be_positive(cls, v):
        if v <= 0:
            raise ValueError('Age must be positive')
        return v
user_0 = User(age=25)
print(user_0)
user_1 = User(age=-25)
print(user_1)

age=25


ValidationError: 1 validation error for User
age
  Value error, Age must be positive [type=value_error, input_value=-25, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

In [42]:
from pydantic import BaseModel, Field, model_validator
from typing import Dict

class Patient(BaseModel):
    name: str
    age: int = Field(..., gt=0)
    weight: float = Field(..., gt=0)
    married: bool
    contact_details: Dict[str, str]  # Includes keys like "emergency_contact", "home", etc.

    @model_validator(mode='after')
    def validate_emergency_contact(self,):
        age = self.age
        contact_details = self.contact_details
        if age > 60 and "emergency_contact" not in contact_details:
            raise ValueError('For patients over 60, "emergency_contact" must be present in contact_details.')
        return self

Patient(
    name="John Doe",
    age=99,
    weight=70.5,
    married=True,
    contact_details={
        "home": "123-456-7890",
    }
)

ValidationError: 1 validation error for Patient
  Value error, For patients over 60, "emergency_contact" must be present in contact_details. [type=value_error, input_value={'name': 'John Doe', 'age...'home': '123-456-7890'}}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

In [43]:
from pydantic import BaseModel, computed_field

class Product(BaseModel):
    name: str
    price: float
    quantity: int

    @computed_field
    @property
    def total_price(self) -> float:
        return self.price * self.quantity

# Usage
p = Product(name="Apple", price=10.5, quantity=3)
print(p.total_price)  # 31.5


31.5
